In [4]:
from funcs_mrcio import iwrhdr_opened, irdhdr_opened, iwrsec_opened, irdpas_opened
from funcs_mrcio import irdsec_opened
import matplotlib.pyplot as plt
import numpy as np

from skimage.exposure import rescale_intensity

In [5]:
micrographnamein= "test_data/test_lattice.mrc"
micrographnameout= micrographnamein.replace(".mrc", "_filt.mrc")
psize = 1.048 # Angstroms/pixel

micrograph = open(micrographnamein,'rb')
image = rescale_intensity(irdsec_opened(micrograph,0)) #rescale to range (-1,1)
header = irdhdr_opened(micrograph)
nx = header['nx']; ny = header['ny']; nz = header['nz']

In [6]:
boxsize = np.maximum(nx,ny)
image_padded=np.empty([boxsize,boxsize])
image_padded[:,:]=np.mean(image)
image_padded[0:nx,0:ny]=image

In [7]:
import pyfftw   
fftmod = pyfftw.interfaces.numpy_fft
pyfftw.interfaces.cache.enable()
import multiprocessing
imagefftw  = pyfftw.empty_aligned([boxsize,boxsize],dtype=np.float64)
imagefftw[:,:]=image_padded
imagec=fftmod.rfftn(imagefftw,threads=1,planner_effort="FFTW_ESTIMATE")
ny_fft, nx_fft= np.shape(imagec)


In [8]:
from skimage import filters
power        = np.log(np.real(imagec * np.conj(imagec)))
power_smooth = filters.gaussian(power,9)
peaky        = power-power_smooth
peaky_smooth = filters.gaussian(peaky,1)

In [9]:
filters.threshold_mean(peaky_smooth)

-0.00460664620970096

In [14]:
np.std(peaky_smooth)

0.3853848838142234

In [11]:
threshold    = 3*np.std(peaky_smooth)+filters.threshold_mean(peaky_smooth) 
masked_index_y, masked_index_x= np.where(peaky_smooth[:, 0:7*np.int(nx_fft/8)]>threshold)
masked= power.copy()
first_half= np.where(masked_index_y<=(ny_fft/2))[0]
second_half= np.where(masked_index_y>(ny_fft/2))[0]
masked_index_y_first= masked_index_y[first_half]
masked_index_y_second= masked_index_y[second_half]

/tmp/ipykernel_32124/162142332.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  masked_index_y, masked_index_x= np.where(peaky_smooth[:, 0:7*np.int(nx_fft/8)]>threshold)


In [39]:
X= np.int(0.01*nx_fft)
Y= np.int(0.01*ny_fft)

/tmp/ipykernel_30981/4165795667.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X= np.int(0.01*nx_fft)
/tmp/ipykernel_30981/4165795667.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more de

In [40]:
masked_index_y_shifted= np.append(masked_index_y_first+Y, masked_index_y_second-Y)
masked_index_x_shifted= masked_index_x+X

In [41]:
masked[masked_index_y, masked_index_x]= power[masked_index_y_shifted, masked_index_x_shifted]

In [44]:
phase_array= np.random.uniform(low= 0.0, high= 2*np.pi, size= np.shape(imagec))
j= 0.0+1j

#copying fourier transformed image
imagecmasked = imagec.copy()
#adding a random phase to the fourier transformed image
imagecmasked_random_phase= imagec.copy()*np.exp(phase_array*j)

#masking out the diffraction spots by replacing the peaks with the values (with added random phase) from the shifted axis
imagecmasked[masked_index_y, masked_index_x]= imagecmasked_random_phase[masked_index_y_shifted,masked_index_x_shifted]

# calculate inverse FFT of masked FTi
image_nolattice_padded=fftmod.irfftn(imagecmasked,threads=1,planner_effort='FFTW_ESTIMATE')

#extract micrograph from the padded, masked
image_nolattice=image_nolattice_padded[0:nx,0:ny]

#plt.rcParams['figure.figsize'] = [20, 40]
#print(np.min(image_nolattice),np.max(image_nolattice))
#plt.imshow(image_nolattice,cmap='gray')
#plt.show()

filteredmic = open(micrographnameout,'wb')
nxyz  = np.array([nx,ny,1],dtype=np.float32)
dmin  = np.min(image_nolattice)
dmax  = np.max(image_nolattice)
dmean = np.sum(image_nolattice)/(nx*ny)
iwrhdr_opened(filteredmic,nxyz,dmin,dmax,dmean,mode=2)
iwrsec_opened(image_nolattice,filteredmic)